# Segment
This notebook contains example code for how you can use the PadAnalsysis package to segment colonies and single cells to produce segmentation masks. If you have a time-series of images, and want to extract statistics, use the next notebook as a guide. 

In [4]:
# Install dependencies with:
# pip install -r requirements.txt

# set up notebook
%load_ext autoreload
%autoreload 2

import os
import natsort
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2 as cv
from PadAnalyser.MicrocolonySegmenter import ZStack, MKSegmentUtils, DInfo, ColonySegment, CellSegment, Segment

IMAGE_FOLDER = 'sample_images'
OUTPUT_FOLDER = 'output'
SPECIES = 'ecoli', 'staph', 'pseu'

In [5]:
# set up folder paths
input_folder = IMAGE_FOLDER 
work_folder = os.path.join(OUTPUT_FOLDER, 'work')
results_folder = os.path.join(OUTPUT_FOLDER, 'results')

# create folders and clear work folder
if not os.path.exists(results_folder): os.makedirs(results_folder)
if not os.path.exists(work_folder): os.makedirs(work_folder)
else: # clear it
    for f in os.listdir(work_folder):
        os.remove(os.path.join(work_folder, f))

def get_all_frames_in_folder(path: str) -> list[tuple[str, str]]:
    filenames = [n for n in natsort.natsorted(os.listdir(path)) if n.endswith('.png')]
    return [(n, os.path.join(path, n)) for n in filenames]

frames = []
labels = []
filenames = []
for s in SPECIES:
    for filename, filepath in get_all_frames_in_folder(os.path.join(input_folder, s)):        
        frame = np.asarray(Image.open(filepath))
        label = f'{s}_{filename.split(".")[0]}'

        frames.append(frame)
        labels.append(label)
        filenames.append(filename)


dinfo = DInfo.DInfo(
    label='seg',
    image_dir=work_folder,
    printing=False,
    live_plot=False,
    file_plot=True,
)
dinfo_results = DInfo.DInfo(
    label='seg',
    image_dir=results_folder,
    printing=True,
    live_plot=False,
    file_plot=True,
)

print(f'Loaded {len(frames)} frames:')
for i, l in enumerate(labels): 
    print(f'\t{i:2d}. {l}')

Loaded 12 frames:
	 0. ecoli_BE145_11_A12_bf_i10_t07954_9511-9519-l9
	 1. ecoli_BE148_02_A3_bf_i06_t05532_8632-8646-l15
	 2. ecoli_BE148_06_A7_bf_i03_t02804_4395-4408-l14
	 3. ecoli_BE151_69_G2_bf_i04_t03575_4331-4339-l9
	 4. ecoli_BE151_69_G2_bf_i08_t06666_7971-7979-l9
	 5. staph_BE205_00_A1_bf_i01_t00625_1067-1078-l12
	 6. staph_BE205_04_A5_bf_i06_t03775_6449-6460-l12
	 7. staph_BE205_06_A7_bf_i03_t01917_3277-3288-l12
	 8. pseu_BE204_22_B1_bf_i05_t03685_6332-6343-l12
	 9. pseu_BE216_11_A12_bf_i10_t07288_12624-12635-l12
	10. pseu_BE216_11_A12_bf_i13_t09477_16368-16379-l12
	11. pseu_BE216_42_D6_bf_i02_t01757_3043-3054-l12


In [6]:
# Run single frames and study performance
for f, l in list(zip(frames, labels))[:]:
    print(f'Processing {l}')    
    
    # load species specific parameters
    species = l.split('_')[0]
    species_full_name = Segment.species_map[species]
    params = Segment.analysis_parameters[species_full_name]

    sigma = params['sigma']
    threshold = params['threshold']
    split_factor = params['split_factor']
    min_mask_size_filter = params['min_mask_size_filter']
    
    # customize output debug info object
    d = dinfo.append_to_label(l).with_live_plot(False).with_file_plot(False)
    d_results = dinfo_results.append_to_label(l).with_live_plot(False).with_file_plot(True)

    # Preprocess frame    
    frame = ZStack.clip(f)
    frame = cv.GaussianBlur(frame, (3, 3), 0)
    frame = MKSegmentUtils.norm(frame)
    
    # perform colony segmentation
    c_contours = ColonySegment.bf_via_edges(frame, dinfo=d.append_to_label('a_contour'))#, lower_threshold=30, upper_threshold=120, close_iterations=3)    

    # perform cell segmentation
    s_contours = CellSegment.bf_laplacian(
        frame, 
        colony_contours=c_contours, 
        dinfo=d.append_to_label(f'cell_sigma{sigma}_threshold{threshold}'), 
        sigma=sigma, 
        ksize=7,
        threshold=threshold,
        split_factor=split_factor,
        min_mask_size_filter=min_mask_size_filter,
    )

    # Plot results
    MKSegmentUtils.plot_frame(frame, dinfo=d_results.append_to_label('res_0_colony'), contours=c_contours, contour_thickness=2)
    MKSegmentUtils.plot_frame(frame, dinfo=d_results.append_to_label('res_1_cells'), contours=s_contours, contour_thickness=cv.FILLED)
    MKSegmentUtils.plot_frame_color_area(frame, dinfo=d_results.append_to_label('res_2_cell_area'), contours=s_contours)

Processing ecoli_BE145_11_A12_bf_i10_t07954_9511-9519-l9
Processing ecoli_BE148_02_A3_bf_i06_t05532_8632-8646-l15
Processing ecoli_BE148_06_A7_bf_i03_t02804_4395-4408-l14
Processing ecoli_BE151_69_G2_bf_i04_t03575_4331-4339-l9
Processing ecoli_BE151_69_G2_bf_i08_t06666_7971-7979-l9
Processing staph_BE205_00_A1_bf_i01_t00625_1067-1078-l12
Processing staph_BE205_04_A5_bf_i06_t03775_6449-6460-l12
Processing staph_BE205_06_A7_bf_i03_t01917_3277-3288-l12
Processing pseu_BE204_22_B1_bf_i05_t03685_6332-6343-l12
Processing pseu_BE216_11_A12_bf_i10_t07288_12624-12635-l12
Processing pseu_BE216_11_A12_bf_i13_t09477_16368-16379-l12
Processing pseu_BE216_42_D6_bf_i02_t01757_3043-3054-l12
